In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from seminartools.models.distance_model import DistanceModel
from seminartools.models.random_effects_model import RandomEffectsModel
from seminartools.models.uc_sv_ss_model import UCSVSSModel
from seminartools.data import read_merged
import pandas as pd

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
df = read_merged().reset_index()
df

c:\users\vandi\onedrive\documenten\github\seminar\seminartools\data.py:161: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df = df.resample("Q").apply(lambda x: (1 + x).prod() - 1)  # quarterly return
c:\users\vandi\onedrive\documenten\github\seminar\seminartools\data.py:397: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df = df.resample("Q").mean()  # Mean interest rate


,country,date,inflation,gdp_growth,interest_rate,unemployment_rate,commodity_CRUDE_PETRO,commodity_iNATGAS,commodity_iAGRICULTURE,commodity_iMETMIN,commodity_iPRECIOUSMET
0,Japan,1977-01-01,0.022951,2.187257,6.333333,2.3,0.00000,0.052425,0.144757,0.041940,0.112740
1,Canada,1977-01-01,0.023904,1.533830,8.166667,8.8,0.00000,0.052425,0.144757,0.041940,0.112740
2,United States,1977-01-01,0.017231,1.186233,4.700000,8.2,0.00000,0.052425,0.144757,0.041940,0.112740
3,Japan,1977-04-01,0.027244,0.703788,5.000000,2.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
4,Canada,1977-04-01,0.023346,0.510302,7.666667,8.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
...,...,...,...,...,...,...,...,...,...,...,...
2166,Italy,2023-01-01,0.003938,0.552944,3.000000,8.3,-0.02041,-0.585458,0.002037,0.007418,0.043183
2167,Canada,2023-01-01,0.006080,0.610026,4.500000,5.4,-0.02041,-0.585458,0.002037,0.007418,0.043183
2168,Sweden,2023-01-01,0.014955,0.656897,2.833333,7.8,-0.02041,-0.585458,0.002037,0.007418,0.043183
2169,Korea,2023-01-01,0.010538,0.329875,3.500000,3.2,-0.02041,-0.585458,0.002037,0.007418,0.043183


In [4]:
distance_model = DistanceModel(pointwise_aggregation_method = "distribution")
distance_model

In [5]:
distance_model.fit(df)

Creating regression coefficients: 100%|██████████| 1/1 [00:02<00:00,  2.41s/it]


In [6]:
preds_dist = distance_model.predict(df)

In [7]:
from seminartools.models.utils import make_oos_predictions
from seminartools.time_series_split import ExpandingWindowSplit

tss = ExpandingWindowSplit(start_date = "2000-01-01")
preds_dist = make_oos_predictions(lambda: DistanceModel(pointwise_aggregation_method="distribution"), df, tss, progress = True, num_cores = 5)


Splits: 100%|██████████| 5/5 [00:00<00:00, 107.28it/s]


In [8]:
preds_dist

,country,inflation,date
0,Australia,"{'pdf': [7.023951852398276e-07, 7.049402299574...",2000-01-01
1,Belgium,"{'pdf': [3.73949012286932e-07, 3.7462610939818...",2000-01-01
2,Canada,"{'pdf': [2.256811867453696e-06, 2.263986542956...",2000-01-01
3,Denmark,"{'pdf': [1.6991379555421433e-06, 1.70434674609...",2000-01-01
4,Italy,"{'pdf': [3.9860368727857894e-07, 3.99471807345...",2000-01-01
...,...,...,...
1648,South Africa,"{'pdf': [3.1176026980046705e-07, 3.11768846664...",2023-01-01
1649,Spain,"{'pdf': [3.0702627693334545e-07, 3.07149321109...",2023-01-01
1650,Sweden,"{'pdf': [3.1781887760484164e-07, 3.18326781951...",2023-01-01
1651,Switzerland,"{'pdf': [4.3322062219633577e-07, 4.33888310122...",2023-01-01


In [9]:
predictions_dist = preds_dist.merge(
        df, on=["date", "country"], suffixes=("_pred", "_true")
    ).dropna()
predictions_dist

,country,inflation_pred,date,inflation_true,gdp_growth,interest_rate,unemployment_rate,commodity_CRUDE_PETRO,commodity_iNATGAS,commodity_iAGRICULTURE,commodity_iMETMIN,commodity_iPRECIOUSMET
0,Australia,"{'pdf': [7.023951852398276e-07, 7.049402299574...",2000-01-01,0.007788,0.410167,5.333333,7.2,0.095219,0.218652,0.002835,0.024545,0.008684
1,Belgium,"{'pdf': [3.73949012286932e-07, 3.7462610939818...",2000-01-01,0.008163,0.711111,3.250000,7.3,0.095219,0.218652,0.002835,0.024545,0.008684
2,Canada,"{'pdf': [2.256811867453696e-06, 2.263986542956...",2000-01-01,0.005405,1.565721,5.000000,7.3,0.095219,0.218652,0.002835,0.024545,0.008684
3,Denmark,"{'pdf': [1.6991379555421433e-06, 1.70434674609...",2000-01-01,0.005333,0.975100,3.583333,5.3,0.095219,0.218652,0.002835,0.024545,0.008684
4,Italy,"{'pdf': [3.9860368727857894e-07, 3.99471807345...",2000-01-01,0.006775,1.229391,3.250000,11.4,0.095219,0.218652,0.002835,0.024545,0.008684
...,...,...,...,...,...,...,...,...,...,...,...,...
1643,South Africa,"{'pdf': [3.1176026980046705e-07, 3.11768846664...",2023-01-01,0.011202,0.406619,7.416667,28.1,-0.020410,-0.585458,0.002037,0.007418,0.043183
1644,Spain,"{'pdf': [3.0702627693334545e-07, 3.07149321109...",2023-01-01,0.006013,0.504719,3.000000,13.3,-0.020410,-0.585458,0.002037,0.007418,0.043183
1645,Sweden,"{'pdf': [3.1781887760484164e-07, 3.18326781951...",2023-01-01,0.014955,0.656897,2.833333,7.8,-0.020410,-0.585458,0.002037,0.007418,0.043183
1646,Switzerland,"{'pdf': [4.3322062219633577e-07, 4.33888310122...",2023-01-01,0.010084,0.309543,1.166667,4.4,-0.020410,-0.585458,0.002037,0.007418,0.043183


In [11]:
from seminartools.model_evaluation import get_brier_score
get_brier_score(predictions_dist)

0.5655461165048643

In [13]:
tss = ExpandingWindowSplit(start_date = "2000-01-01")
preds_dist = make_oos_predictions(lambda: DistanceModel(pointwise_aggregation_method="distribution"), df, tss, progress = True, num_cores = 5)
preds_ucsv = make_oos_predictions(lambda: UCSVSSModel(num_particles=10000, stochastic_seasonality=True,pointwise_aggregation_method="distribution"), df, tss, progress = True, num_cores = 5)
preds_re = make_oos_predictions(lambda: RandomEffectsModel(pointwise_aggregation_method="distribution"), df, tss, progress = True, num_cores = 5)
predictions_dist = preds_dist.merge(
    df, on=["date", "country"], suffixes=("_pred", "_true")
).dropna()
predictions_ucsv = preds_ucsv.merge(
    df, on=["date", "country"], suffixes=("_pred", "_true")
).dropna()
predictions_re = preds_re.merge(
    df, on=["date", "country"], suffixes=("_pred", "_true")
).dropna()
brier_dist = get_brier_score(predictions_dist)
brier_ucsv = get_brier_score(predictions_ucsv)
brier_re = get_brier_score(predictions_re)

   
    

Splits: 100%|██████████| 5/5 [00:00<00:00, 539.02it/s]


Fitting model on the full dataset... (model=<seminartools.models.uc_sv_ss_model.UCSVSSModel object at 0x000001F04EC4A1D0>)





100%|██████████| 20/20 [00:02<00:00,  8.41it/s]


Fitted!


Splits: 100%|██████████| 5/5 [00:00<00:00, 261.13it/s]


In [14]:
brier_frame = pd.DataFrame(data = [brier_ucsv, brier_dist, brier_re], columns = ["brier_score"], index = ["ucsv", "dist", "re"])
brier_frame

,brier_score
ucsv,0.606476
dist,0.569369
re,0.579745
